# 单例模式

__new__是在实例创建之前被调用的，因为它的任务就是创建实例然后返回该实例对象，是个静态方法。
__init__是当实例对象创建完成后被调用的，然后设置对象属性的一些初始值，通常用在初始化一个类实例的时候。是一个实例方法。

In [4]:
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls,'instance'):
            cls.instance = super().__new__(cls)
        return cls.instance
    
s = Singleton()
print("Object created", s)

s1 = Singleton()
print("Object created", s1)

Object created <__main__.Singleton object at 0x0000023EF9316BE0>
Object created <__main__.Singleton object at 0x0000023EF9316BE0>


# 懒汉式实例化

使用@staticmethod或@classmethod，就可以不需要实例化，直接类名.方法名()来调用。
既然@staticmethod和@classmethod都可以直接类名.方法名()来调用，那他们有什么区别呢
从它们的使用上来看,
@staticmethod不需要表示自身对象的self和自身类的cls参数，就跟使用函数一样。
@classmethod也不需要self参数，但第一个参数需要是表示自身类的cls参数。
如果在@staticmethod中要调用到这个类的一些属性方法，只能直接类名.属性名或类名.方法名。
而@classmethod因为持有cls参数，可以来调用类的属性，类的方法，实例化对象等，避免硬编码。

In [13]:
class Singleton():
    _instance = None
    def __init__(self):
        if not self._instance:
            print("__init__ method called..")
        else:
            print("Instance already created:", self.getInstance())
            
    @classmethod
    def getInstance(cls):
        if not cls._instance:
            cls._instance = Singleton()
        return cls._instance

s = Singleton()
print("Object created s ", s)
s2 = Singleton()
print("Object created s2 " , s2)
print("Object created 1", Singleton.getInstance())
print("Object created 2", Singleton.getInstance())
s1 = Singleton()
print("Object created s1 ", s1)


__init__ method called..
Object created s  <__main__.Singleton object at 0x0000023EF9474908>
__init__ method called..
Object created s2  <__main__.Singleton object at 0x0000023EF95E3D68>
__init__ method called..
Object created 1 <__main__.Singleton object at 0x0000023EF95D50B8>
Object created 2 <__main__.Singleton object at 0x0000023EF95D50B8>
Instance already created: <__main__.Singleton object at 0x0000023EF95D50B8>
Object created s1  <__main__.Singleton object at 0x0000023EF95EA0B8>


# Monostate 单例模式

上一篇文章中，我们讨论的是GOF编写的设计模式图书中的相关内容。GoF的单例设计模式是指，一个类有且只有一个对象。
然而，根据Alex Martelli的说法，通常程序员需要的是让实例共享相同的状态。他建议开发人员更应关注状态和行为，而不是同一性。
由于该概率基于所有对象共享相同的状态，因此被称为Monostate（单态）模式。

如下程序，我们将变量__shared_state赋给__dict__（python用__dict__存储一个类所有对象的状态），
这样，这个类创建的所有实例都通过__shared_state共享状态。

In [6]:
class Borg:
    __shared_state={"1":"2"}
    def __init__(self):
        self.x = 1
        self.__dict__ = self.__shared_state
        pass

b = Borg()
b1 = Borg()
b.x = 4

print(b)
print(b1)
print(b.__dict__)
print(b1.__dict__)


{'1': '2', 'x': 4}
{'1': '2', 'x': 4}


In [8]:
class Borg(object):
    _shared_state = {}
    def __new__(cls):
        obj = super().__new__(cls)
        obj.__dict__ = cls._shared_state
        return obj
b2 = Borg()
b3 = Borg()
b2.x = 4
print(b2)
print(b3)
print(b2.__dict__)
print(b3.__dict__)

{'x': 4}
{'x': 4}


# 单例和元类

In [16]:
class MyInt(type):
    def __call__(cls, *args, **kwds):
        return (type.__call__(cls, *args, **kwds))

class int(metaclass=MyInt):
    def __init__(self, x, y):
        self.x = x
        self.y = y

i = int(4,5)
print(i)

TypeError: __init__() takes 3 positional arguments but 4 were given